In [1]:
from vsm import LoadDataset, VectorSpaceModel

## Load the dataset
Functions to load the dataset and the given queries with associated relevant documents.

In [2]:
dataset = LoadDataset("./med_data/MED.ALL", "./med_data/MED.QRY", "./med_data/MED.REL")
docs = dataset.load_docs()
queries = dataset.load_queries()
relevance = dataset.load_relevance()